# Tugas Eksplorasi DAA 1

Author: Eduardus Tjitrahardja 2106653602


## Helper


In [ ]:
%pip install memory-profiler

In [16]:
import time
from memory_profiler import memory_usage
import random


# Function to perform a sorting operation and measure time
def run_algo_and_measure(algo, *args):
    start_time = time.time()
    # Replace this line with your sorting algorithm of choice
    result = algo(*args)
    end_time = time.time()
    execution_time = end_time - start_time
    # convert to millisecondsq
    execution_time *= 1000
    return execution_time, result


def evaluate(algo, *args) -> None:
    start_mem = memory_usage()[0]
    execution_time, result = run_algo_and_measure(algo, *args)
    end_mem = memory_usage()[0]
    mem_usage = end_mem - start_mem
    if type(result) == tuple:
        result = result[0]
    print(f"Result: {result}")
    print(f"Execution time: {execution_time:.6f} ms; Memory usage: {mem_usage} MiB")


def generate_items(n: int) -> list[int]:
    prices = [random.randint(1, 100) for _ in range(n)]
    weights = [random.randint(1, 100) for _ in range(n)]
    return prices, weights


def save_dataset_to_txt(arr: list[int], path: str) -> None:
    with open(path, "w") as f:
        for num in arr:
            f.write(f"{num}\n")


def load_dataset_from_txt(path: str) -> list[int]:
    with open(path, "r") as f:
        return [int(line) for line in f.readlines()]

## 0/1 Unbounded Knapsack


In [6]:
def unboundedKnapsack(W, val, wt):
    # dp[i] is going to store maximum
    # value with knapsack capacity i.
    dp = [0] * (W + 1)

    # Fill dp[] using above recursive formula
    for i in range(W + 1):
        for j in range(len(val)):
            if wt[j] <= i:
                dp[i] = max(dp[i], dp[i - wt[j]] + val[j])

    return dp[W]


# Driver program
W = 100
prices = [10, 30, 20, 40]
weights = [5, 10, 15, 12]

print("Optimal value:", unboundedKnapsack(W, prices, weights))

Optimal value: 320


In [7]:
evaluate(unboundedKnapsack, W, prices, weights)

Result: 320
Execution time: 0.000000 ms; Memory usage: 0.0 MiB


## An improved branch and bound algorithm for a strongly correlated unbounded knapsack problem


In [12]:
import numpy as np


class BranchBoundUKP:
    def __init__(self, W, w, v):
        self.W = W
        self.w = w
        self.v = v
        self.N = list(range(len(w)))
        self.M = None
        self.x_best = None
        self.z_best = 0

    def calculate_max_value(self):
        # Procedure 1: Eliminating dominated items
        j = 0
        while j < len(self.N) - 1:
            k = j + 1
            while k < len(self.N):
                wj, wk = self.w[self.N[j]], self.w[self.N[k]]
                vj, vk = self.v[self.N[j]], self.v[self.N[k]]
                if np.floor(wk / wj) * vj >= vk:
                    self.N.pop(k)
                elif np.floor(wj / wk) * vk >= vj:
                    self.N.pop(j)
                    k = len(self.N)
                else:
                    k += 1
            j += 1

        # Procedure 2: Proposed algorithm
        # Initialize
        self.N.sort(key=lambda i: self.v[i] / self.w[i], reverse=True)
        self.v = [self.v[i] for i in self.N]
        self.w = [self.w[i] for i in self.N]

        self.x_best = np.zeros(len(self.N), dtype=int)  # current best solution
        self.z_best = 0  # current best solution value
        x_current = np.zeros(len(self.N), dtype=int)  # current feasible solution

        self.M = np.empty((len(self.N), len(self.N)))

        i = 0
        x1 = int(np.floor(self.W / self.w[i]))
        W_prime = self.W - self.w[i] * x1
        V_N = self.v[i] * x1
        U = self.calculate_U(W_prime, V_N, i)
        m = []
        for i in range(len(self.N)):
            min_mi = min([self.w[j] for j in range(i + 1, len(self.N))], default=np.inf)
            m.append(min_mi)

        # branching
        command = 2
        while True:
            if command == 2:
                i, x_current, W_prime, V_N, command = self.develop(
                    i, x_current, W_prime, V_N, U, m
                )
            elif command == 3:
                i, x_current, W_prime, V_N, command = self.backtrack(
                    i, x_current, W_prime, V_N, m
                )
            elif command == 4:
                ...  # not used
            else:
                break

        return self.z_best, self.x_best

    def develop(
        self,
        i,
        x_current,
        W_prime,
        V_N,
        U,
        m,
    ):
        # Develop
        while True:
            if W_prime < m[i]:
                if self.z_best < V_N:
                    self.z_best = V_N
                    self.x_best = np.copy(x_current)
                    if self.z_best == U:
                        return (i, x_current, W_prime, V_N, 5)
                return (i, x_current, W_prime, V_N, 3)
            else:
                j = self.find_j(W_prime, i)
                if (V_N + U[j, int(W_prime)] <= z_best) or (
                    self.M[i, int(W_prime)] >= V_N
                ):
                    return (i, x_current, W_prime, V_N, 3)
                else:
                    x_current[j] = int(np.floor(W_prime / self.w[j]))
                    V_N += self.v[j] * x_current[j]
                    W_prime -= self.w[j] * x_current[j]
                    self.M[i, int(W_prime)] = V_N
                    i = j

    def backtrack(
        self,
        i,
        x_current,
        W_prime,
        V_N,
        m,
    ):
        while True:
            # Backtrack
            j = self.find_max_j(x_current, i)
            if j < 0:
                return (i, x_current, W_prime, V_N, 5)
            i = j
            x_current[i] -= 1
            V_N -= self.v[i]
            W_prime += self.w[i]

            if W_prime < m[i]:
                continue
            elif (
                V_N + np.floor(W_prime * (self.v[i + 1] / self.w[i + 1])) <= self.z_best
            ):
                V_N -= self.v[i] * x_current[i]
                W_prime += self.w[i] * x_current[i]
                x_current[i] = 0
            elif W_prime >= m[i]:
                return (i, x_current, W_prime, V_N, 2)

    def calculate_U(self, W_prime, V_N, i):
        if i >= len(self.w) - 2:
            return V_N

        v1, v2, v3 = self.v[i], self.v[i + 1], self.v[i + 2]
        w1, w2, w3 = self.w[i], self.w[i + 1], self.w[i + 2]
        z_prime = V_N + (np.floor(W_prime / w2) * v2)
        W_double_prime = W_prime - (np.floor(W_prime / w2) * w2)
        U_prime = z_prime + (np.floor(W_double_prime / w3) * v3)
        U_double_prime = z_prime + (
            np.floor(
                (
                    (W_double_prime + (np.ceil((1 / w1) * (w2 - W_double_prime)) * w1))
                    * (v2 / w2)
                )
                - (np.ceil((1 / w1) * (w2 - W_double_prime)) * v1)
            )
        )

        return max(U_prime, U_double_prime)

    def find_j(self, W_prime, i):
        j = i + 1
        while j < len(self.w) and self.w[j] > W_prime:
            j += 1
        return j

    def find_max_j(self, x_current, i):
        j = i if i < len(x_current) else len(x_current) - 1
        while j >= 0 and x_current[j] == 0:
            j -= 1
        return j


# Example usage:
W = 100
prices = [10, 30, 20, 40]
weights = [5, 10, 15, 12]

bbukp = BranchBoundUKP(W, weights, prices)
z_best, x_best = bbukp.calculate_max_value()

print("Optimal value:", z_best)
print("Optimal solution vector:", x_best)

Optimal value: 320
Optimal solution vector: [0 0 0]


In [24]:
bbukp = BranchBoundUKP(W, weights, prices)
evaluate(bbukp.calculate_max_value)

Result: 320
Execution time: 0.000000 ms; Memory usage: 0.0 MiB


## Generate Dataset


In [4]:
kecil = generate_items(100)
sedang = generate_items(1000)
besar = generate_items(10000)

In [17]:
save_dataset_to_txt(kecil[0], "v_kecil.txt")
save_dataset_to_txt(sedang[0], "v_sedang.txt")
save_dataset_to_txt(besar[0], "v_besar.txt")

save_dataset_to_txt(kecil[1], "w_kecil.txt")
save_dataset_to_txt(sedang[1], "w_sedang.txt")
save_dataset_to_txt(besar[1], "w_besar.txt")

In [18]:
v_kecil = load_dataset_from_txt("v_kecil.txt")
w_kecil = load_dataset_from_txt("w_kecil.txt")

v_sedang = load_dataset_from_txt("v_sedang.txt")
w_sedang = load_dataset_from_txt("w_sedang.txt")

v_besar = load_dataset_from_txt("v_besar.txt")
w_besar = load_dataset_from_txt("w_besar.txt")

## Evaluation


In [20]:
print("Unbounded Knapsack")
print("Kecil")
evaluate(unboundedKnapsack, 10, kecil[0], kecil[1])
print("Sedang")
evaluate(unboundedKnapsack, 10, sedang[0], sedang[1])
print("Besar")
evaluate(unboundedKnapsack, 10, besar[0], besar[1])

Unbounded Knapsack
Kecil
Result: 310
Execution time: 0.000000 ms; Memory usage: 0.0 MiB
Sedang
Result: 980
Execution time: 0.999928 ms; Memory usage: 0.0 MiB
Besar
Result: 1000
Execution time: 7.517338 ms; Memory usage: 0.0 MiB


In [28]:
print("Branch and Bound Unbounded Knapsack")
print("Kecil")
bbukp = BranchBoundUKP(10, w_kecil, v_kecil)
evaluate(bbukp.calculate_max_value)
print("Sedang")
bbukp = BranchBoundUKP(10, w_sedang, v_sedang)
evaluate(bbukp.calculate_max_value)
print("Besar")
bbukp = BranchBoundUKP(10, w_besar, v_besar)
evaluate(bbukp.calculate_max_value)

Branch and Bound Unbounded Knapsack
Kecil
Result: 310
Execution time: 0.000000 ms; Memory usage: 0.0 MiB
Sedang
Result: 980
Execution time: 2.001762 ms; Memory usage: 0.0 MiB
Besar
Result: 1000
Execution time: 19.539356 ms; Memory usage: 0.0 MiB
